detect_symbol 项目是一个目标检测任务：从电气图纸中找到并标出某些客户关心的类型的符号。我们对这个项目的定位是“借此项目来练习目标检测任务乃至深度学习任务的开发流程”，因此我们格外注意代码组织、函数封装，思路分析，过程记录和经验总结。本文是对整个开发流程的记录和复盘，除了对各设计模块做原理和设计说明，还会记录一些知识点和过程中的思考。

# 总述

从客户给的数张电气图纸开始，到最终形成一个能够部署的服务，这个过程的各个部分列出如下：

1. 获取数据
- 理解数据
- 生成训练数据
- 数据组织为databunch
- 设计模型
- 设计grid和anchor
- 设计loss和metrics
- 设计优化器
- 训练过程规划
- 训练过程可视化
- 训练
- 分析训练结果
- 推理

下面将对这些步骤逐个说明。

# 获取数据

深度学习项目的第一步是获取数据，数据的质和量是深度学习项目能否成功的关键。

在detect_symbol项目中，客户给了数张电子版的电气设计图，我们把它转换成了图片。目标检测任务需要带标注的图片，我们自己做了手工标注。

客户说明了它们关心哪些类别的符号，我们根据这些信息对电气图纸（图片）进行了手动标注，过程如下：
- 建立一个word文档，列出关心的符号类别，并列出了各类别的符号的样例，方便我们对照标注。
- 我们以windows自带的“画图”软件来做标注
    - 紧贴每个目标符号边界画一个矩形框
    - 严格规定矩形框边框宽度
    - 以不同颜色区别不同类别的目标，颜色与类别的对应关系也列在了上述文档中
    - 标注完图片后要保存为png格式，因为其它（压缩）格式会（轻微）改变我们绘制的矩形框的像素值。
- 根据矩形框提取坐标和类别信息
    - 以opencv来提取信息
    - 以kernel匹配来定位矩形框的左上角和右下角并提取其坐标
    - 以矩形框的像素值来提取类别信息
    - 将原图文件名、坐标信息和类别信息保存在csv文件中
- 检查标注结果
    - 编写代码，根据csv文件中的标注信息，在原图上绘制边框并打印对应类别，并另存
    - 人肉检查：
        - 验证提取信息的算法是否正确
        - 检查人工标注是否有误
    - 错误更正
        - 检查到有错误的标注时，回到原人工标注的图片做更正
        - 以白色将错误边框覆盖，然后绘制新的标注框
        - 再次运行提取坐标和类别信息的代码
- 关于错误标注的说明：
    - 深度学习需要大量的数据，有监督学习还需要标注信息
    - 标注信息难免有误，据传imagenet里就有数十万的错误标注
    - 少量的，随机的误标注对深度学习训练的影响是非常小的，因此只需做到尽量检查但不必过于纠结
    - 但是有偏的误标注是可能产生危害的，要避免
    - 何谓“随机误标注”和“有偏误标注”，我不能准确定义，下面以例子说明
        - 随机误标注：
            - 标注框有时（稍微）偏左，有时偏右，有时偏大，有时偏小
            - 少量目标的类别标为其它的随机类别。
        - 有偏误标注：
            - 所有标注框都偏右
            - 所有1类都标为3类

经过这个过程，我们有了原图，有了与每个原图上每个目标符号对应的边框和类别信息，它们记录在csv文件中。

要说明的是，人工标框的图片要保存为png格式，目的是严格的保持矩形框的像素值，但是原图可以是jpg（或其他压缩）格式，可以节省硬盘空间。

还要说明的是，人肉标注耗时耗力，庆幸的是detect_symbol项目的原图和符号的数量不算大，这仍然花费了两个人半天时间。现在有一些自动化标注的网页，可以对你的图片做粗标注或辅助标注，提高人肉标注的效率，我还没有尝试过，但值得关注。另一种选择是雇佣数据标注公司，这又是土豪干的事情了，跟我们公司没有关系。

# 理解数据